In [0]:
import pandas as pd
import os
import io
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from google.colab import files
import csv
import keras
from keras.datasets import mnist
from keras.layers import Dense # Dense layers are "fully connected" layers
from keras.models import Sequential # Documentation: https://keras.io/models/sequential/
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import img_to_array, load_img
import math
from keras.models import clone_model

Using TensorFlow backend.


In [0]:
from google.colab import files
uploaded = files.upload()

Saving sample.csv to sample.csv
Saving train.csv to train.csv


In [0]:
#Load in CSV
trainingLabelsDf = pd.read_csv(io.StringIO(uploaded['train.csv'].decode('utf-8')))
print(trainingLabelsDf.head())
trainingImageNames = trainingLabelsDf['Id'].values
dgci = trainingLabelsDf['DGCI'].values

         Id      DGCI
0  41367835 -0.320235
1  56194592 -0.322221
2  75713836 -0.312795
3   1169576 -0.312034
4  88155343 -0.318707


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
imageNames = ['/content/drive/My Drive/Challenge6Pictures/' + str(imageName) + '.jpg' for imageName in trainingImageNames]
images = [img_to_array(load_img(imageName)) for imageName in imageNames]
for i in range(len(images)):
  if images[i].shape[0] != 300:
    images[i] = np.swapaxes(images[i], 0, 1)
  if images[i].shape[0] != 300:
    print(i)
imagesArray = np.stack(images)
imagesArray = imagesArray / 255

In [0]:
# labels = [np.mean(image) for image in images]
labels = np.array(dgci)

In [0]:
xTrain, xTest, yTrain, yTest = train_test_split(imagesArray, labels, test_size=0.2, random_state=22)
xTrain = np.array(xTrain)
xTest = np.array(xTest)
yTrain = np.array(yTrain)
yTest = np.array(yTest)

In [0]:
xTrain

array([[[[0.22352941, 0.30588236, 0.2784314 ],
         [0.26666668, 0.33333334, 0.30980393],
         [0.30588236, 0.34901962, 0.33333334],
         ...,
         [0.3764706 , 0.41960785, 0.40392157],
         [0.4       , 0.44705883, 0.43137255],
         [0.40392157, 0.4509804 , 0.43529412]],

        [[0.23921569, 0.3137255 , 0.2901961 ],
         [0.2627451 , 0.32156864, 0.3019608 ],
         [0.3137255 , 0.35686275, 0.34117648],
         ...,
         [0.36862746, 0.4117647 , 0.39607844],
         [0.42352942, 0.47058824, 0.45490196],
         [0.39607844, 0.44313726, 0.42745098]],

        [[0.28235295, 0.34117648, 0.32156864],
         [0.23137255, 0.2784314 , 0.2627451 ],
         [0.27450982, 0.30980393, 0.29803923],
         ...,
         [0.33333334, 0.3764706 , 0.36078432],
         [0.39607844, 0.44313726, 0.42745098],
         [0.4117647 , 0.45882353, 0.44313726]],

        ...,

        [[0.30588236, 0.30980393, 0.32941177],
         [0.2627451 , 0.26666668, 0.28627452]

In [0]:
#loss = ?

activation = 'relu'

model = Sequential()

model.add(
    Conv2D(
        32, 
        (3, 3), 
        input_shape=(300, 400, 3),
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(
    Conv2D(
        64, 
        (3, 3), 
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(
    Conv2D(
        128, 
        (3, 3), 
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(
    Conv2D(
        256, 
        (3, 3), 
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(
    Dense(
        256,
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)

model.add(
    Dense(
        256,
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)

model.add(
    Dense(
        256,
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)

model.add(
    Dense(
        256,
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('linear'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# model.compile(optimizer="sgd", loss='mean_squared_error')
model.compile(
    loss='mean_squared_error',
    optimizer='adam'
)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 298, 398, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 149, 199, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 147, 197, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 98, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 71, 96, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 35, 48, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 33, 46, 256)       295168    
__________

In [0]:
batchSize = 32
epochs = 1

In [0]:
history = model.fit(xTrain, yTrain, batch_size=batchSize, epochs=epochs, verbose=True, validation_split=.2)
loss  = model.evaluate(xTest, yTest, verbose=False)

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['training', 'validation'], loc='best')
# axes = plt.gca()
# axes.set_ylim([0,1])
# plt.show()
print('Test loss:', loss)

# # serialize model to JSON
# model_json = model.to_json()
# with open("model2.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model2.h5")
# files.download("model2.json")
# files.download("model2.h5")

Train on 506 samples, validate on 127 samples
Epoch 1/1
506/506 [==============================] - 198s 390ms/step - loss: 0.0369 - val_loss: 0.0201
Test loss: 0.0071223775018592304


In [0]:
modelClone = clone_model(model)
modelClone.set_weights(model.get_weights())
modelClone.compile(
    loss='mean_squared_error',
    optimizer='adam'
)


In [0]:
loss  = modelClone.evaluate(xTest, yTest, verbose=False)
print('Test loss:', loss)

Test loss: 0.002664298274548455


In [0]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
files.download("model.json")
files.download("model.h5")

In [0]:
predictions = model.predict(xTest)

In [0]:
predictions

array([[0.44406682],
       [0.4168896 ],
       [0.46972734],
       [0.27567822],
       [0.3628314 ],
       [0.16819213],
       [0.42056736],
       [0.05986269],
       [0.47476673],
       [0.4419973 ],
       [0.4663577 ],
       [0.4718803 ],
       [0.46673083],
       [0.4442697 ],
       [0.36424693],
       [0.06050726],
       [0.4766723 ],
       [0.3535375 ],
       [0.47779953],
       [0.40727574],
       [0.06129736],
       [0.32859612],
       [0.44742903],
       [0.2959129 ],
       [0.48484132],
       [0.4744128 ],
       [0.45905742],
       [0.05973777],
       [0.47609207],
       [0.35007513],
       [0.33073992],
       [0.4637224 ],
       [0.4705488 ],
       [0.38673395],
       [0.48063818],
       [0.3275297 ],
       [0.2829217 ],
       [0.35623235],
       [0.4677001 ],
       [0.46822163],
       [0.47273022],
       [0.39093205],
       [0.3861319 ],
       [0.36757246],
       [0.0733557 ],
       [0.47502044],
       [0.38194305],
       [0.059

In [0]:
for i in range(25):
  print(predictions[i])
  print(yTest[i])

[0.46977237]
0.49637957
[0.45180887]
0.491033671
[0.4906141]
0.479137621
[0.28484863]
0.506618177
[0.3899267]
0.48545090700000004
[0.22554216]
0.40151612799999997
[0.4534964]
0.49409270899999996
[-0.4908211]
-0.353876532
[0.48743322]
0.553653472
[0.4685035]
0.493582206
[0.48659304]
0.507568053
[0.49574623]
0.489306756
[0.4821393]
0.514728105
[0.47315714]
0.479377358
[0.38777515]
0.487707264
[-0.48497143]
-0.297001938
[0.49461737]
0.496054481
[0.3856817]
0.538715995
[0.49695668]
0.493125576
[0.44246876]
0.46105624
[-0.4794232]
-0.342479155
[0.3710389]
0.5382328829999999
[0.47303644]
0.502669583
[0.3225896]
0.48690932299999995
[0.49676105]
0.533759885


In [0]:
mse = ((predictions.flatten() - yTest)**2).mean(axis=0)
print(mse)
print(math.sqrt(mse))


0.011010060567805822
0.10492883573072667


In [0]:
#Load in CSV
testingLabelsDf = pd.read_csv(io.StringIO(uploaded['sample.csv'].decode('utf-8')))
print(testingLabelsDf.head())
testingImageNames = testingLabelsDf['Id'].values

         Id  DGCI
0  79783930     0
1  40239934     0
2  50410534     0
3  47165338     0
4  16455157     0


In [0]:
testImageNames = ['/content/drive/My Drive/Challenge6Pictures/' + str(imageName) + '.jpg' for imageName in testingImageNames]
testImages = [img_to_array(load_img(imageName)) for imageName in testImageNames]
position = 0
for i in range(len(testImages)):
  if testImages[i].shape[0] != 300:
    testImages[i] = np.swapaxes(testImages[i], 0, 1)
  if testImages[i].shape[0] != 300:
    print(i)
  position += 1
testImagesArray = np.stack(testImages)
testImagesArray = testImagesArray / 255

In [0]:
testPredictions = model.predict(testImagesArray)

In [0]:
with open("Turf Predictions 6.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerows(testPredictions)
files.download("Turf Predictions 6.csv")

In [0]:
#https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

#loss = 0.05238204971893028

activation = 'sigmoid'

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(300, 400, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation(activation))

model.add(Conv2D(128, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(64, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation(activation))
model.add(Dropout(0.5))
model.add(Dense(16))
model.add(Activation(activation))
model.add(Dense(1))
model.add(Activation('linear'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

model.compile(optimizer="adam", loss='mse', metrics=['accuracy'])

model.summary()

In [0]:
#https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

#loss = ?

activation = 'sigmoid'

model = Sequential()
model.add(Conv2D(64, (5, 5), input_shape=(300, 400, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (5, 5)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(128))
model.add(Activation(activation))

model.add(Dense(64))
model.add(Activation(activation))

model.add(Dense(32))
model.add(Activation(activation))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('relu'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

model.compile(optimizer="adam", loss='mse')

model.summary()

In [0]:
#loss = 0.05079054610096125

activation = 'sigmoid'

model = Sequential()


model.add(Conv2D(16, (3, 3), input_shape=(300, 400, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))
model.add(Activation(activation))

model.add(Dense(64))
model.add(Activation(activation))

model.add(Dense(32))
model.add(Activation(activation))
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('linear'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

model.compile(optimizer="sgd", loss='mse')

model.summary()

In [0]:
#DOESNT WORK

activation = 'relu'

model = Sequential()


model.add(Conv2D(16, (3, 3), input_shape=(300, 400, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(8, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(4, (3, 3)))
model.add(Activation(activation))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(16))
model.add(Activation(activation))
model.add(Dropout(0.5))

model.add(Dense(8))
model.add(Activation(activation))

model.add(Dense(4))
model.add(Activation(activation))

model.add(Dense(1))
model.add(Activation('linear'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

model.compile(optimizer="adam", loss='mse')

model.summary()

In [0]:
#predicted different values!!
#loss = ?.049 or something

activation = 'relu'

model = Sequential()

model.add(
    Conv2D(
        32, 
        (3, 3), 
        input_shape=(300, 400, 3),
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(
    Conv2D(
        64, 
        (3, 3), 
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(
    Dense(
        128,
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)

model.add(
    Dense(
        128,
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)

model.add(
    Dense(
        128,
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)

model.add(
    Dense(
        128,
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('linear'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# model.compile(optimizer="sgd", loss='mean_squared_error')
model.compile(
    loss='mean_squared_error',
    optimizer='rmsprop'
)
model.summary()

In [0]:
#best so far!!
#loss = .0249 or something

activation = 'relu'

model = Sequential()

model.add(
    Conv2D(
        32, 
        (3, 3), 
        input_shape=(300, 400, 3),
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(
    Conv2D(
        64, 
        (3, 3), 
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(
    Conv2D(
        128, 
        (3, 3), 
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(
    Conv2D(
        256, 
        (3, 3), 
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(
    Dense(
        256,
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)

model.add(
    Dense(
        256,
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)

model.add(
    Dense(
        256,
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)

model.add(
    Dense(
        256,
        activation=activation,
        kernel_initializer='random_uniform',
        bias_initializer='zeros'
    )
)
model.add(Dropout(0.5))

model.add(Dense(1))
model.add(Activation('linear'))

# model.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

# model.compile(optimizer="sgd", loss='mean_squared_error')
model.compile(
    loss='mean_squared_error',
    optimizer='adam'
)
model.summary()